In [ ]:
import pandas as pd

# 📌 Rutas de los archivos
final_df_path = r"C:\Users\solan\Downloads\get_data_from_songs\src\final_df.csv"
correcciones_path = r"C:\Users\solan\Downloads\get_data_from_songs\src\errores_canciones_corregidos.csv"
output_path = r"C:\Users\solan\Downloads\get_data_from_songs\src\final_df_corregido.csv"

# 📌 Cargar los datasets
df = pd.read_csv(final_df_path)
df_correcciones = pd.read_csv(correcciones_path)

# 📌 Convertir recording_id a string (por seguridad)
df["recording_id"] = df["recording_id"].astype(str)
df_correcciones["recording_id"] = df_correcciones["recording_id"].astype(str)

# 📌 Verificar que todas las columnas coincidan en ambos archivos
if not set(df.columns).issuperset(df_correcciones.columns):
    missing_cols = set(df_correcciones.columns) - set(df.columns)
    raise ValueError(f"❌ ERROR: Las siguientes columnas están en el archivo de correcciones pero no en final_df.csv: {missing_cols}")

# 📌 Crear un diccionario con las correcciones {recording_id: {columna: valor}}
correcciones_dict = df_correcciones.set_index("recording_id")[["spotify_url", "processed_lyrics"]].to_dict(orient="index")

# 📌 Validación y actualización segura
modificaciones = 0
errores = []

for index, row in df.iterrows():
    rec_id = row["recording_id"]

    if rec_id in correcciones_dict:
        # 📌 Guardar valores originales antes de actualizar
        original_values = row.copy()

        # 📌 Aplicar correcciones solo en las columnas permitidas
        row["spotify_url"] = correcciones_dict[rec_id]["spotify_url"]
        row["processed_lyrics"] = correcciones_dict[rec_id]["processed_lyrics"]

        # 📌 Validar que solo se modificaron las columnas esperadas
        for col in df.columns:
            if col not in ["spotify_url", "processed_lyrics"] and row[col] != original_values[col]:
                errores.append((rec_id, col))
        
        # 📌 Si la validación es exitosa, aplicar cambios
        if not errores:
            df.loc[index] = row  # ✅ Guardar cambios en el DataFrame
            modificaciones += 1
        else:
            print(f"⚠️ ERROR: Se detectaron cambios en otras columnas para recording_id {rec_id}. No se aplicó la corrección.")

# 📌 Guardar el nuevo dataset corregido
df.to_csv(output_path, index=False)
print(f"✅ {modificaciones} canciones corregidas con éxito. Archivo guardado en:\n{output_path}")

if errores:
    print(f"❌ {len(errores)} errores detectados donde otras columnas cambiaron accidentalmente.")


In [ ]:
import pandas as pd

# 📌 Rutas de los archivos
original_file = r"C:\Users\solan\Downloads\get_data_from_songs\src\final_df.csv"
correcciones_file = r"C:\Users\solan\Downloads\get_data_from_songs\src\errores_canciones_corregidos.csv"
output_file = r"C:\Users\solan\Downloads\get_data_from_songs\src\final_df_actualizado.csv"
backup_file = r"C:\Users\solan\Downloads\get_data_from_songs\src\final_df_backup.csv"

# 📌 Cargar datasets
df_original = pd.read_csv(original_file)
df_correcciones = pd.read_csv(correcciones_file)

# 📌 Copia de seguridad antes de modificar el dataset original
df_original.to_csv(backup_file, index=False)
print(f"📌 Copia de seguridad creada en {backup_file}")

# 📌 Fusionar datasets basados en recording_id (sin duplicar)
df_actualizado = df_original.merge(df_correcciones, on="recording_id", how="left", suffixes=("", "_corregido"))

# 📌 Aplicar correcciones solo en las filas con valores corregidos
for col in ["spotify_url", "processed_lyrics"]:  # 🛠️ Puedes añadir más columnas si necesitas
    df_actualizado[col] = df_actualizado[col + "_corregido"].combine_first(df_actualizado[col])

# 📌 Eliminar columnas temporales de fusión
df_actualizado.drop(columns=[col + "_corregido" for col in ["spotify_url", "processed_lyrics"]], inplace=True)

# 📌 Guardar dataset actualizado
df_actualizado.to_csv(output_file, index=False)

print(f"✅ Dataset actualizado guardado en {output_file}")
print("🚀 No hay duplicados, solo se reemplazaron datos corregidos.")
